## Import dependencies

In [ ]:
# import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
from pathlib import Path
from skimage.color import rgb2hsv
from PIL import Image
from numpy import asarray, save
from sklearn import preprocessing

In [ ]:
# load pre-trained model from specified location for faster labeling
model = torch.hub.load('ultralytics/yolov5', 'custom', path=os.path.join("yolov5", "runs", "train", "exp(mAP.5:.05:.95)", "weights", "best.pt"), force_reload=True)

In [ ]:
# Specify images and labels path
new_images_path = os.path.join('new_label_data', 'images')
new_labels_path = os.path.join('new_label_data', 'labels')

## Use pre-trained model to annotate images

In [ ]:
# Function to get predicted label & bounding box coordinates back from model

def get_yolo_annotations(fil):
    
    im = cv2.imread(fil)
    
    h, w, _ = im.shape
    
    results=model(im)
    results = results.pandas().xyxy[0]
    results.drop(['confidence', 'name'], axis=1, inplace=True)
    results["width"]  = (results["xmax"] - results["xmin"])/w
    results["height"] = (results["ymax"] - results["ymin"])/h
    results["x"] = (results["xmin"] + ((results["xmax"] - results["xmin"])/2))/w
    results["y"] = (results["ymin"] + ((results["ymax"] - results["ymin"])/2))/h
    results = results[["class", "x", "y", "width", "height"]]
    
    return results

In [ ]:
# Predicting objects for every image that needs to be labeled
# Saving results according to YOLO format in labels folder

for fil in new_images_path.glob("*.jpg"):
    
    annotations = get_yolo_annotations(fil)
    
    new_label_file = ".\\" + str(new_labels_path) + "\\" + fil.stem + ".txt"
    new_image_file = ".\\" + str(fil)
    
    print(annotations.values)
    
    annotations.to_csv(new_label_file, header=None, index=None, sep=' ', mode='a', float_format='%.6f')

## Opening LabelImg, an annotation program

In [ ]:
!labelImg